#Google Cloud interactions with Python

## Big Query
We can access BQ tables with Google's libraries. We must use Google Oauth Authentication for that purpose.

For more information about Oauth2, check https://oauth.net/2/.

We can either use a **Service Account** or the Google Cloud Auth library.

### With Google Big Query libraries
Let see a few actions you can easily do with GBQ libraries.

Let's check if a Dataset exists.

In [ ]:
from google.cloud import bigquery
from google.api_core.exceptions import NotFound
from google.oauth2 import service_account

# use your own credentials
json_credential_file_path = "savvy-camp-345623-27fe83e29978.json"
credentials = service_account.Credentials.from_service_account_file(
  json_credential_file_path,
  scopes = ["https://www.googleapis.com/auth/cloud-platform"]
)
project_id = "savvy-camp-345623"
dataset_id = "data_2023"
table_id = "sample_timestamps"

client = bigquery.Client(credentials = credentials, project = project_id)

# remove try / except and change names
try:
  client.get_dataset(dataset_id)
  print("Dataset {} already exists".format(dataset_id))
except NotFound:
  print("Dataset {} is not found".format(dataset_id))


###With Colab Auth libraries

In [ ]:
from google.colab import auth
from google.cloud import bigquery
from google.api_core.exceptions import NotFound

auth.authenticate_user()

project_id = "savvy-camp-345623"
dataset_id = "data_2023"
client = bigquery.Client(project = project_id)

# remove try / except and change names
try:
  client.get_dataset(dataset_id)
  print("Dataset {} already exists".format(dataset_id))
except NotFound:
  print("Dataset {} is not found".format(dataset_id))


###Tables, rows, SQL...
Let's insert a record in a table.

In [ ]:
import datetime

try:
  # understand format() method
  table_path = "{}.{}.{}".format(project_id, dataset_id, table_id)
  table = client.get_table(table_path)
  rows = [
           {
             "who": "GCP - Collab - 20260115 Python!",
             "when": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
           }
         ]

  # this is asyncronous!
  errors = client.insert_rows_json(table, rows)
  if errors == []:
    print("Success")
  else:
    print(errors)

except Exception as e:
  print(e)

Let's bring a whole table, row by row.

In [ ]:
try:
  dataset = client.get_dataset(dataset_id)
  table = dataset.table(table_id)

  rows = client.list_rows(table)
  print(rows.total_rows, 'rows:\n')
  for row in rows:
    print(row)

except Exception as e:
  print(e)

Finally, let's run a query.

In [ ]:
try:
  sql = "select * from `{}.{}.{}` where who like 'GCP%005%'".format(project_id, dataset_id, table_id)
  job = client.query(sql)
  rows = job.result()
  print(rows.total_rows, 'rows:\n')
  for row in rows:
    print(row)

except Exception as e:
  print(e)

You can find a full set of Python GBQ samples at https://cloud.google.com/bigquery/docs/samples.

###With Pandas
As you may have noticed, dealing directly with query results is not very handy. However, they can easily be transformed into _pandas_ dataframes.

In [ ]:
import pandas as pd

try:
  dataset = client.get_dataset(dataset_id)
  table = dataset.table(table_id)

  df = client.list_rows(table).to_dataframe()
  print(df)

except Exception as e:
  print(e)

In [ ]:
df

In [ ]:
try:
  sql = "select * from `{}.{}.{}` where who like 'GCP%005%'".format(project_id, dataset_id, table_id)
  job = client.query(sql)

  df = job.result().to_dataframe()
  print(df)

except Exception as e:
  print(e)

### Colab DataTable
Colab provides some libraries to deal with data. DataTable is a powerful tool to visualize data in a tabular way.

In [ ]:
from google.colab import data_table
data_table.DataTable(df, include_index = False, num_rows_per_page = 10)

###Question
Where would you run a ``group by``? In the database (via query) on in your Python code (via pandas)?

Let's check https://mode.com/blog/group-by-sql-python/!



## Google Analytics


###GA4 and Dataframes


In [ ]:
!pip install google-analytics-data

from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import DateRange
from google.analytics.data_v1beta.types import Dimension
from google.analytics.data_v1beta.types import Metric
from google.analytics.data_v1beta.types import RunReportRequest
from google.oauth2 import service_account

import pandas as pd
from datetime import datetime

def create_ga4_client(file_name):
  credentials_ga4 = service_account.Credentials.from_service_account_file(
      file_name,
      scopes = ['https://www.googleapis.com/auth/analytics.readonly']
  )
  return BetaAnalyticsDataClient(credentials = credentials_ga4)

def ga4_response_to_df(response):
  data = {}
  for d in response.dimension_headers:
    data[d.name] = []
  for m in response.metric_headers:
    data[m.name] = []

  for row in response.rows:
    for i, d in enumerate(response.dimension_headers):
      if d.name == 'date':
        data[d.name].append(datetime.strptime(row.dimension_values[i].value, "%Y%m%d").date())
      else:
        data[d.name].append(row.dimension_values[i].value)
    for i, m in enumerate(response.metric_headers):
      data[m.name].append(int(row.metric_values[i].value))

  df = pd.DataFrame(data)
  return df

In [ ]:
print('Start')
print('Client')
client = create_ga4_client("ga4isd.json")
print('Request')
request = RunReportRequest(
  property = 'properties/241325456',
  dimensions = [Dimension(name = "date"), Dimension(name = "country")],
  metrics = [Metric(name = "totalUsers")],
  date_ranges = [DateRange(start_date = "2026-01-01", end_date = "2026-01-14")]
)
print('Run Report')
response = client.run_report(request)
# print('Response')
# print(response)
print('Dataframe')
ga4_response_to_df(response)

###Let's play
Add metrics, dimensions, filters...

## All together
As you may have notice, data from GA is not really manageable and BQ data in tabular form is not very handy. So let's build a few of programs to add some value to all this code.


### Sessions timeline


In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

def plot_chart(df, cold, colv):
  plt.figure(figsize = (16, 10), dpi = 80)
  plt.plot(cold, colv, data = df.sort_values(by = 'date'), color = 'red')

  # Decoration
  plt.xticks(rotation = 0, fontsize = 12, horizontalalignment = 'center', alpha = .7)
  plt.yticks(fontsize = 12, alpha = .7)
  plt.title("Sessions in 2025", fontsize = 22)
  plt.grid(axis = 'both', alpha = .3)

  # Borders
  plt.gca().spines["top"].set_alpha(0.0)
  plt.gca().spines["bottom"].set_alpha(0.3)
  plt.gca().spines["right"].set_alpha(0.0)
  plt.gca().spines["left"].set_alpha(0.3)
  plt.show()

# Main program
client = create_ga4_client("ga4isd.json")
request = RunReportRequest(
  property = 'properties/241325456',
  dimensions = [Dimension(name = "date")],
  metrics = [Metric(name = "sessions")],
  date_ranges = [DateRange(start_date = "2023-01-01", end_date = "2026-01-10")]
)
response = client.run_report(request)
df = ga4_response_to_df(response)
plot_chart(df, 'date', 'sessions')

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

def plot_chart(df, col_date, col_value, col_predicted, col_lower, col_upper):
  plt.figure(figsize = (16, 10), dpi = 80)
  plt.plot(df.index, col_predicted, data = df, color = 'orange')
  plt.plot(df.index, col_lower, data = df, color = 'red')
  plt.plot(df.index, col_upper, data = df, color = 'green')
  plt.plot(df.index, col_value, data = df, color = 'blue')

  # Decoration
  plt.xticks(rotation = 0, fontsize = 12, horizontalalignment = 'center', alpha = .7)
  plt.yticks(fontsize = 12, alpha = .7)
  plt.title("Sessions in 2025", fontsize = 22)
  plt.grid(axis = 'both', alpha = .3)

  # Borders
  plt.gca().spines["top"].set_alpha(0.0)
  plt.gca().spines["bottom"].set_alpha(0.3)
  plt.gca().spines["right"].set_alpha(0.0)
  plt.gca().spines["left"].set_alpha(0.3)
  plt.show()

df2 = df.set_index('date')
model = ARIMA(df2['sessions'], order = (1, 1, 1))
results = model.fit()
pred = results.get_prediction(start = df2.index[0], end = df2.index[-1])
conf_int = pred.conf_int(alpha = 0.05) # 95% confidence
df2['predicted'] = pred.predicted_mean
df2['lower'] = conf_int.iloc[:, 0]
df2['upper'] = conf_int.iloc[:, 1]
plot_chart(df2.sort_index(), 'date', 'sessions', 'predicted', 'lower', 'upper')

##Compare weekly pageviews in some countries
Even more complex!
Let's compare pageviews during the first and second week of May in Belgium, Germany, Portugal, United Kingdom and France.

In [ ]:
import matplotlib.lines as mlines
import seaborn as sns

def plot_chart_newline(p1, p2, color = 'black'):
  ax = plt.gca()
  l = mlines.Line2D([p1[0], p2[0]], [p1[1], p2[1]], color = 'red' if p1[1] > p2[1] else 'green', marker = 'o', markersize = 6)
  ax.add_line(l)
  return l

def plot_chart(df, col1, col2, coli):
  fig, ax = plt.subplots(1, 1, figsize = (14, 14), dpi = 80)

  # Vertical Lines
  ax.vlines(x = 1, ymin = 0, ymax = 20000, color = 'black', alpha = 0.7, linewidth = 1, linestyles = 'dotted')
  ax.vlines(x = 3, ymin = 0, ymax = 20000, color = 'black', alpha = 0.7, linewidth = 1, linestyles = 'dotted')

  # Points
  ax.scatter(y = df[col1], x = np.repeat(1, df.shape[0]), s = 10, color = 'black', alpha = 0.7)
  ax.scatter(y = df[col2], x = np.repeat(3, df.shape[0]), s = 10, color = 'black', alpha = 0.7)

  # Line Segmentsand Annotation
  for p1, p2, c in zip(df[col1], df[col2], df[coli]):
    plot_chart_newline([1, p1], [3, p2])
    ax.text(1 - 0.05, p1, c + ', ' + str(round(p1)), horizontalalignment = 'right', verticalalignment ='center', fontdict = {'size': 14})
    ax.text(3 + 0.05, p2, c + ', ' + str(round(p2)), horizontalalignment = 'left', verticalalignment = 'center', fontdict = {'size': 14})

  # Date Annotations
  ax.text(1-0.05, 19500, col1, horizontalalignment = 'right', verticalalignment = 'center', fontdict = {'size': 18, 'weight': 700})
  ax.text(3+0.05, 19500, col2, horizontalalignment = 'left', verticalalignment = 'center', fontdict = {'size': 18, 'weight': 700})

  # Decoration
  ax.set_title("Compare pageviews: {} vs {}".format(col1, col2), fontdict = {'size':22})
  ax.set(xlim = (0, 4), ylim = (0, 14000), ylabel = 'Page Views')
  ax.set_xticks([1, 3])
  ax.set_xticklabels([col1, col2])
  plt.yticks(np.arange(0, 20001, 5000), fontsize = 12)

  # Lighten borders
  plt.gca().spines["top"].set_alpha(.50)
  plt.gca().spines["bottom"].set_alpha(.50)
  plt.gca().spines["right"].set_alpha(.50)
  plt.gca().spines["left"].set_alpha(.50)
  plt.show()


# Main program
client = create_ga4_client("ga4isd.json")
request_2025 = RunReportRequest(
  property = 'properties/241325456',
  dimensions = [Dimension(name = "country")],
  metrics = [Metric(name = "screenPageViews")],
  date_ranges = [DateRange(start_date = "2025-01-01", end_date = "2025-01-10")]
)
response_2025 = client.run_report(request_2025)
df_2025 = ga4_response_to_df(response_2025)
request_2026 = RunReportRequest(
  property = 'properties/241325456',
  dimensions = [Dimension(name = "country")],
  metrics = [Metric(name = "screenPageViews")],
  date_ranges = [DateRange(start_date = "2026-01-01", end_date = "2026-01-10")]
)
response_2026 = client.run_report(request_2026)
df_2026 = ga4_response_to_df(response_2026)

df_2025.rename(columns = {'screenPageViews': '10D 2025'}, inplace = True)
df_2026.rename(columns = {'screenPageViews': '10D 2026'}, inplace = True)
df_all_countries = pd.merge(df_2025, df_2026, on = "country")
df = df_all_countries[df_all_countries['country'].isin(['Belgium', 'Germany', 'Bulgaria', 'United Kingdom', 'France'])]

plot_chart(df, '10D 2025', '10D 2026', 'country')